# convnets

Instantiating a small convnet

a convnet takes as input tensors of shape (image_height, image_width,
image_channels), not including the batch dimension.

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 128)         7385

Because we’re doing 10-way classification with a
softmax output, we’ll use the categorical crossentropy loss, and because our labels are
integers, we’ll use the sparse version, sparse_categorical_crossentropy.

Training the convnet on MNIST images

In [6]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype("float32") / 255
model.compile(optimizer="rmsprop",
loss="sparse_categorical_crossentropy",
metrics=["accuracy"])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 13s 13ms/step - loss: 0.1571 - accuracy: 0.9513
Epoch 2/5
938/938 [==============================] - 8s 8ms/step - loss: 0.0442 - accuracy: 0.9864
Epoch 3/5
938/938 [==============================] - 5s 6ms/step - loss: 0.0312 - accuracy: 0.9901
Epoch 4/5
938/938 [==============================] - 6s 6ms/step - loss: 0.0232 - accuracy: 0.9926
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 0.0185 - accuracy: 0.9946


Evaluating the convnet

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 3s 9ms/step - loss: 0.0277 - accuracy: 0.9914
Test accuracy: 0.991


##The convolution operation

In Conv2D layers, padding is configurable via the padding argument, which takes two
values: "valid", which means no padding (only valid window locations will be used),
and "same", which means “pad in such a way as to have an output with the same width
and height as the input.” The padding argument defaults to "valid"

Using stride 2 means the width and height of the feature map are downsampled by a
factor of 2 (in addition to any changes induced by border effects)

Strided convolutions are rarely used in classification models, but they come in handy for some types of
models,

 the role of max pooling: to aggressively downsample feature maps, much like
strided convolutions

A big difference from convolution is that max pooling is usually done
with 2 × 2 windows and stride 2, in order to downsample the feature maps by a factor of 2. On the other hand, convolution is typically done with 3 × 3 windows and no
stride (stride 1).

An incorrectly structured convnet missing its max-pooling layers

In [8]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation="softmax")(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

In [9]:
model_no_max_pool.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 conv2d_8 (Conv2D)           (None, 22, 22, 128)       73856     
                                                                 
 flatten_2 (Flatten)         (None, 61952)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                619530    
                                                                 
Total params: 712,202
Trainable params: 712,202
Non-trainab

The final feature map has 22 × 22 × 128 = 61,952 total coefficients per sample.
This is huge. When you flatten it to stick a Dense layer of size 10 on top, that
layer would have over half a million parameters. This is far too large for such a
small model and would result in intense overfitting.

In short, the reason to use downsampling is to reduce the number of feature-map
coefficients to process, as well as to induce spatial-filter hierarchies by making successive convolution layers look at increasingly large windows (in terms of the fraction of
the original input they cover).

The most reasonable subsampling strategy is to first produce dense maps of
features (via unstrided convolutions) and then look at the maximal activation of the
features over small patches, rather than looking at sparser windows of the inputs (via
strided convolutions) or averaging input patches, which could cause you to miss or
dilute feature-presence information.

##Training a convnet from scratch on a small dataset

these three strategies—training a small model from scratch,
doing feature extraction using a pretrained model, and fine-tuning a pretrainedmodel—will constitute your future toolbox for tackling the problem of performing
image classification with small datasets.

First, you need to create a Kaggle API key and download it to your local machine. Just
navigate to the Kaggle website in a web browser, log in, and go to the My Account
page. In your account settings, you’ll find an API section. Clicking the Create New API
Token button will generate a kaggle.json key file and will download it to your machine.
Second, go to your Colab notebook, and upload the API’s key JSON file to your Colab
session by running the following code in a notebook cell:

In [10]:
#from google.colab import files
#files.upload()

In [11]:
#mkdir ~/.kaggle

In [12]:
#cp kaggle.json ~/.kaggle/

In [13]:
#!chmod 600 ~/.kaggle/kaggle.json

In [14]:
#!kaggle competitions download -c dogs-vs-cats

Finally, create a ~/.kaggle folder (mkdir ~/.kaggle), and copy the key file to it
(cp kaggle.json ~/.kaggle/). As a security best practice, you should also make
sure that the file is only readable by the current user, yourself (chmod 600):
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
You can now download the data we’re about to use:
!kaggle competitions download -c dogs-vs-cats
The first time you try to download the data, you may get a “403 Forbidden” error.
That’s because you need to accept the terms associated with the dataset before you
download it—you’ll have to go to www.kaggle.com/c/dogs-vs-cats/rules (while
logged into your Kaggle account) and click the I Understand and Accept button. You
only need to do this once.

Finally, the training data is a compressed file named train.zip. Make sure you uncompress it (unzip) silently (-qq):

In [15]:
#!unzip -qq dogs-vs-cats.zip

In [16]:
#!unzip -qq train.zip

Copying images to training, validation, and test directories

In [17]:
import os, shutil, pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

In [18]:
def make_subset(subset_name, start_index, end_index):
  for category in ("cat", "dog"):
    dir = new_base_dir / subset_name / category
    os.makedirs(dir)
    fnames = [f"{category}.{i}.jpg"
      for i in range(start_index, end_index)]
    for fname in fnames:
      shutil.copyfile(src=original_dir / fname,
        dst=dir / fname)
#make_subset("train", start_index=0, end_index=1000)
#make_subset("validation", start_index=1000, end_index=1500)
#make_subset("test", start_index=1500, end_index=2500)

Here, because we start from inputs of size 180 pixels × 180 pixels (a somewhat arbitrary
choice), we end up with feature maps of size 7 × 7 just before the Flatten layer

NOTE The depth of the feature maps progressively increases in the model
(from 32 to 256), whereas the size of the feature maps decreases (from 180 ×
180 to 7 × 7). This is a pattern you’ll see in almost all convnets

Because we’re looking at a binary-classification problem, we’ll end the model with a
single unit (a Dense layer of size 1) and a sigmoid activation. This unit will encode the
probability that the model is looking at one class or the other.

Instantiating a small convnet for dogs vs. cats classification

In [19]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [20]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 89, 89, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                       

For the compilation step, we’ll go with the RMSprop optimizer, as usual. Because we
ended the model with a single sigmoid unit, we’ll use binary crossentropy as the loss

Configuring the model for training

In [21]:
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

###Data preprocessing

As you know by now, data should be formatted into appropriately preprocessed floatingpoint tensors before being fed into the model. Currently, the data sits on a drive as
JPEG files, so the steps for getting it into the model are roughly as follows:
1 Read the picture files.
2 Decode the JPEG content to RGB grids of pixels.
3 Convert these into floating-point tensors.
4 Resize them to a shared size (we’ll use 180 × 180).
5 Pack them into batches (we’ll use batches of 32 images)

Calling image_dataset_from_directory(directory)  will create and return a
tf.data.Dataset object configured to read these files, shuffle them, decode them to
tensors, resize them to a shared size, and pack them into batches

Using image_dataset_from_directory to read images

In [22]:
from tensorflow.keras.utils import image_dataset_from_directory
train_dataset = image_dataset_from_directory(
new_base_dir / "train",
image_size=(180, 180),
batch_size=32)
validation_dataset = image_dataset_from_directory(
new_base_dir / "validation",
image_size=(180, 180),
batch_size=32)
test_dataset = image_dataset_from_directory(
new_base_dir / "test",
image_size=(180, 180),
batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


Displaying the shapes of the data and labels yielded by the Dataset

In [23]:
for data_batch, labels_batch in train_dataset:
  print("data batch shape:", data_batch.shape)
  print("labels batch shape:", labels_batch.shape)
  break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32,)


Fitting the model using a Dataset

In [25]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="convnet_from_scratch.keras",
save_best_only=True,
monitor="val_loss")
]
history = model.fit(
train_dataset,
epochs=30,
validation_data=validation_dataset,
callbacks=callbacks)

Epoch 1/30


NotFoundError: Graph execution error:

Detected at node 'model_3/conv2d_9/Relu' defined at (most recent call last):
    File "c:\Users\GIGABYTE\anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\GIGABYTE\anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\GIGABYTE\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\Users\GIGABYTE\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\Users\GIGABYTE\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\Users\GIGABYTE\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "c:\Us

Displaying curves of loss and accuracy during training

In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined

 Evaluating the model on the test set

In [ ]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

NameError: name 'keras' is not defined

Because we have relatively few training samples (2,000), overfitting will be our
number one concern. You already know about a number of techniques that can help
mitigate overfitting, such as dropout and weight decay (L2 regularization). We’re now
going to work with a new one, specific to computer vision and used almost universally
when processing images with deep learning models: data augmentation

###Using data augmentation

Define a data augmentation stage to add to an image model

In [ ]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)

Displaying some randomly augmented training images

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
for i in range(9):
augmented_images = data_augmentation(images)
ax = plt.subplot(3, 3, i + 1)
plt.imshow(augmented_images[0].numpy().astype("uint8"))
plt.axis("off")

this may not be enough
to completely get rid of overfitting. To further fight overfitting, we’ll also add a Dropout
layer to our model right before the densely connected classifier.

ust
like Dropout, they’re inactive during inference (when we call predict() or evaluate()).
During evaluation, our model will behave just the same as when it did not include
data augmentation and dropout

Defining a new convnet that includes image augmentation and dropout

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

Training the regularized convnet

In [ ]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="convnet_from_scratch_with_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]
history = model.fit(
train_dataset,
epochs=100,
validation_data=validation_dataset,
callbacks=callbacks)

The validation accuracy ends up consistently in the 80–85% range—
a big improvement over our first try.

Evaluating the model on the test set

In [ ]:
test_model = keras.models.load_model(
"convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

By further tuning the model’s configuration (such as the number of filters per
convolution layer, or the number of layers in the model), we might be able to get an
even better accuracy, likely up to 90%. But it would prove difficult to go any higher
just by training our own convnet from scratch, because we have so little data to work
with. As a next step to improve our accuracy on this problem, we’ll have to use a pretrained model

##Leveraging a pretrained model

We’ll use the VGG16 architecture, developed by Karen Simonyan and Andrew
Zisserman in 2014.1 Although it’s an older model, far from the current state of the art
and somewhat heavier than many other recent models, I chose it because its architecture is similar to what you’re already familiar with, and it’s easy to understand without
introducing any new concepts.

There are two ways to use a pretrained model: feature extraction and fine-tuning.
We’ll cover both of them. Let’s start with feature extraction.

###Feature extraction with a pretrained model

Feature extraction consists of using the representations learned by a previously
trained model to extract interesting features from new samples. These features are
then run through a new classifier, which is trained from scratch.

convnets used for image classification comprise two parts:
they start with a series of pooling and convolution layers, and they end with a densely
connected classifier. The first part is called the convolutional base of the model. In the
case of convnets, feature extraction consists of taking the convolutional base of a previously trained network, running the new data through it, and training a new classifier
on top of the output

Why only reuse the convolutional base? Could we reuse the densely connected
classifier as well? In general, doing so should be avoided. The reason is that the representations learned by the convolutional base are likely to be more generic and, therefore, more reusable: the feature maps of a convnet are presence maps of generic
concepts over a picture, which are likely to be useful regardless of the computer vision
problem at hand. But the representations learned by the classifier will necessarily be
specific to the set of classes on which the model was trained—they will only contain
information about the presence probability of this or that class in the entire picture.
Additionally, representations found in densely connected layers no longer contain any information about where objects are located in the input image; these layers get rid of
the notion of space, whereas the object location is still described by convolutional feature maps. For problems where object location matters, densely connected features
are largely useless

In this case, because the ImageNet class set contains multiple dog and cat
classes, it’s likely to be beneficial to reuse the information contained in the densely
connected layers of the original model. But we’ll choose not to, in order to cover
the more general case where the class set of the new problem doesn’t overlap the
class set of the original model.

Let’s put this into practice by using the convolutional base of the VGG16 network, trained on ImageNet, to extract interesting features from cat and dog images, and then train a dogs-versus-cats classifier on top of
these features.

**Instantiating the VGG16 convolutional base**

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
weights="imagenet",
include_top=False,
input_shape=(180, 180, 3))

We pass three arguments to the constructor:
 weights specifies the weight checkpoint from which to initialize the model.
 include_top refers to including (or not) the densely connected classifier on
top of the network. By default, this densely connected classifier corresponds to
the 1,000 classes from ImageNet. Because we intend to use our own densely
connected classifier (with only two classes: cat and dog), we don’t need to
include it.
 input_shape is the shape of the image tensors that we’ll feed to the network.
This argument is purely optional: if we don’t pass it, the network will be able to
process inputs of any size. Here we pass it so that we can visualize (in the following summary) how the size of the feature maps shrinks with each new convolution and pooling layer.

In [ ]:
conv_base.summary()

FAST FEATURE EXTRACTION WITHOUT DATA AUGMENTATION
We’ll start by extracting features as NumPy arrays by calling the predict() method of
the conv_base model on our training, validation, and testing datasets.

Extracting the VGG16 features and corresponding labels

In [ ]:
import numpy as np
def get_features_and_labels(dataset):
  all_features = []
  all_labels = []
  for images, labels in dataset:
    preprocessed_images = keras.applications.vgg16.preprocess_input(images)
    features = conv_base.predict(preprocessed_images)
    all_features.append(features)
    all_labels.append(labels)
  return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)
test_features, test_labels = get_features_and_labels(test_dataset)

the VGG16 model expects
inputs that are preprocessed with the function keras.applications.vgg16.preprocess_input, which scales pixel values to an appropriate range

In [ ]:
train_features.shape

Defining and training the densely connected classifier

In [ ]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="feature_extraction.keras",
save_best_only=True,
monitor="val_loss")
]
history = model.fit(
train_features, train_labels,
epochs=20,
validation_data=(val_features, val_labels),
callbacks=callbacks)

Plotting the results

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

the plots also indicate that we’re overfitting almost from the start—
despite using dropout with a fairly large rate. That’s because this technique doesn’t
use data augmentation, which is essential for preventing overfitting with small image
datasets.

**FEATURE EXTRACTION TOGETHER WITH DATA AUGMENTATION**

Now let’s review the second technique I mentioned for doing feature extraction,
which is much slower and more expensive, but which allows us to use data augmentation during training: creating a model that chains the conv_base with a new dense
classifier, and training it end to end on the inputs

In order to do this, we will first freeze the convolutional base.

In Keras, we freeze a layer or model by setting its trainable attribute to False.

 Instantiating and freezing the VGG16 convolutional base

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
weights="imagenet",
include_top=False)
conv_base.trainable = False

Setting trainable to False empties the list of trainable weights of the layer or model

Printing the list of trainable weights before and after freezing

In [ ]:
conv_base.trainable = True

In [ ]:
print("This is the number of trainable weights "
"before freezing the conv base:", len(conv_base.trainable_weights))

In [ ]:
conv_base.trainable = False

In [ ]:
print("This is the number of trainable weights "
"after freezing the conv base:", len(conv_base.trainable_weights))

we can create a new model that chains together
1 A data augmentation stage
2 Our frozen convolutional base
3 A dense classifier

Adding a data augmentation stage and a classifier to the convolutional base

In [ ]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

This technique is expensive enough that you should only attempt it if
you have access to a GPU (such as the free GPU available in Colab)—it’s
intractable on CPU. If you can’t run your code on GPU, then the previous
technique is the way to go.

In [ ]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="feature_extraction_with_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]
history = model.fit(
train_dataset,
epochs=50,
validation_data=validation_dataset,
callbacks=callbacks)

Let’s plot the results again (see figure 8.14). As you can see, we reach a validation
accuracy of over 98%. This is a strong improvement over the previous model.

Let’s check the test accuracy

Evaluating the model on the test set

In [ ]:
test_model = keras.models.load_model(
"feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

###Fine-tuning a pretrained model

Fine-tuning consists of unfreezing
a few of the top layers of a frozen model base used
for feature extraction, and jointly training both the
newly added part of the model (in this case, the
fully connected classifier) and these top layers. This
is called fine-tuning because it slightly adjusts the
more abstract representations of the model being
reused in order to make them more relevant for the
problem at hand.

the steps for fine-tuning a network are as follows:
1 Add our custom network on top of an
already-trained base network.
2 Freeze the base network.
3 Train the part we added.
4 Unfreeze some layers in the base network.
(Note that you should not unfreeze “batch
normalization” layers, which are not relevant
here since there are no such layers in VGG16.
Batch normalization and its impact on finetuning is explained in the next chapter.)
5 Jointly train both these layers and the part we
added.

In [ ]:
conv_base.summary()

Why not fine-tune more layers? Why not fine-tune the entire convolutional base?
You could. But you need to consider the following:
 Earlier layers in the convolutional base encode more generic, reusable features,
whereas layers higher up encode more specialized features. It’s more useful to
fine-tune the more specialized features, because these are the ones that need
to be repurposed on your new problem. There would be fast-decreasing returns
in fine-tuning lower layers.
 The more parameters you’re training, the more you’re at risk of overfitting.
The convolutional base has 15 million parameters, so it would be risky to
attempt to train it on your small dataset.

Freezing all layers until the fourth from the last

In [ ]:
onv_base.trainable = True
for layer in conv_base.layers[:-4]:
layer.trainable = False

Now we can begin fine-tuning the model. We’ll do this with the RMSprop optimizer,
using a very low learning rate. The reason for using a low learning rate is that we want to
limit the magnitude of the modifications we make to the representations of the three
layers we’re fine-tuning. Updates that are too large may harm these representations.

Fine-tuning the model

In [ ]:
model.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
metrics=["accuracy"])
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="fine_tuning.keras",
save_best_only=True,
monitor="val_loss")
]
history = model.fit(
train_dataset,
epochs=30,
validation_data=validation_dataset,
callbacks=callbacks)

In [ ]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

On the positive side, by leveraging modern deep learning techniques, we managed
to reach this result using only a small fraction of the training data that was available
for the competition (about 10%). There is a huge difference between being able to
train on 20,000 samples compared to 2,000 samples!

Convnets are the best type of machine learning models for computer vision
tasks. It’s possible to train one from scratch even on a very small dataset, with
decent results.
 Convnets work by learning a hierarchy of modular patterns and concepts to
represent the visual world.
 On a small dataset, overfitting will be the main issue. Data augmentation is a
powerful way to fight overfitting when you’re working with image data.
 It’s easy to reuse an existing convnet on a new dataset via feature extraction.
This is a valuable technique for working with small image datasets.
 As a complement to feature extraction, you can use fine-tuning, which adapts to
a new problem some of the representations previously learned by an existing
model. This pushes performance a bit further.